In [1]:
import arcpy
import equations

In [2]:
arcpy.env.workspace = "C:/Users/rucke/SpeedHumpProject/Speed Bump.gdb"
fc = "SpeedHump/SpeedBump"
edit = arcpy.da.Editor(arcpy.env.workspace)

## Accident History Flag

In [3]:
fields = ["TEYA", "FYA", "THYA", "AH"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[3] = equations.acc_hist_flag(row[0], row[1], row[2])
        print("E: ", row[3])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

E:  Hi 10 yr crash
E:  Recent crashes
E:  
E:  Hi 10 yr crash
E:  Hi 5 yr crash
E:  Hi 5 yr crash
E:  Hi 10 yr crash
E:  Hi 5 yr crash
E:  
E:  
E:  Hi 5 yr crash
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  
E:  Hi 5 yr crash
E:  Hi 10 yr crash
E:  Hi 5 yr crash
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  
E:  Hi 10 yr crash
E:  
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  Hi 5 yr crash
E:  
E:  
E:  
E:  
E:  Hi 10 yr crash
E:  
E:  
E:  
E:  
E:  
E:  
E:  
E:  
E:  
E:  Hi 10 yr crash
E:  
E:  
E:  
E:  
E:  Recent crashes
E:  
E:  
E:  
E:  
E:  Hi 10 yr crash
E:  
E:  
E:  
E:  
E:  Hi 10 yr crash
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  
E:  Hi 10 yr crash
E:  
E:  Hi 5 yr crash
E:  
E:  
E:  
E:  Hi 5 yr crash
E:  
E:  
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  
E:  
E:  Hi 10 yr crash
E:  Hi 5 yr crash
E:  
E:  
E:  
E:  


## Pre Score

In [4]:
fields = ["THYA", "FYA", "DA", "SR", "BR", "Park", "OSF", "SWI", "SL", "SW", "CDP", "PS"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        sf_int = []
        for sf in row[2:7]:
            sf_int.append(1 if sf == "Yes" else 0)

        sw = True if row[9] == "Yes" else False
        cdp = True if row[10] == "Yes" else False
        row[11] = float(equations.pre_score(row[0], row[1], sf_int, row[7], row[8], sw, cdp))
        print(row[11] - 0.5)
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

32.151
10.013
4.711
4.159
8.326
10.302
9.76
8.120000000000001
4.068
6.744
9.346
4.0120000000000005
7.504999999999999
14.666
4.675
5.366
36.717
43.729
20.729
13.475
2.1
16.633
16.019
8.05
8.001
23.64
14.0
6.929
16.675
3.6579999999999995
3.9000000000000004
16.647
32.721000000000004
7.646000000000001
3.061
16.749
7.0649999999999995
4.014
4.009
5.167
9.73
1.137
4.163
1.299
3.315
2.991
4.264
4.359999999999999
0.944
2.255
11.998
6.748
0.7389999999999999
1.6640000000000001
4.675
10.36
3.9859999999999998
5.694
1.401
1.004
9.927
6.886
2.0
2.666
5.0
20.719
2.198
15.34
13.65
4.645
15.692
4.663
10.048
1.7269999999999999
1.027
3.7249999999999996
17.999
1.505
4.359999999999999
6.788
23.836
4.159
1.365
0.8380000000000001
30.631
9.884
5.366
6.118
1.173
5.664
10.0
0.9279999999999999
1.019
3.21
12.917
18.628
10.556000000000001
6.2940000000000005
6.65
4.339
0.931
1.149
4.208
3.774
1.164
1.7999999999999998
3.277
3.0
6.0
5.979
1.435
0.9339999999999999
10.6
7.118
14.014
7.0
10.97
7.9830000000000005
1.452
8.

## Speed Calculation

In [5]:
fields = ["FN_E_S", "FS_W_S", "SN_E_S", "SS_W_S_", "CL85"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[4] = float(equations.calc_speed(row[0], row[1], row[2], row[3]))
        print(row[4])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

24.829999923706055
32.619998931884766
32.8799991607666
29.704999923706055
33.05000114440918
32.35000038146973
35.829999923706055
34.2549991607666
39.84499931335449
33.1200008392334
33.02499961853027
36.185001373291016
43.29999923706055
37.75
33.959999084472656
24.5
32.14999961853027
38.55500030517578
41.314998626708984
36.83500099182129
0.0
35.53999900817871
32.5
34.970001220703125
33.5
39.2549991607666
34.170000076293945
27.399999618530273
39.619998931884766
29.0
25.100000381469727
30.199999809265137
27.200000762939453
37.68000030517578
35.619998931884766
27.899999618530273
26.350000381469727
34.59000015258789
31.0
22.350000381469727
23.09999942779541
37.45000076293945
33.5
38.5
24.449999809265137
29.460000038146973
32.654998779296875
40.8650016784668
32.45499897003174
38.045000076293945
27.5
50.52000045776367
30.449999809265137
30.450000762939453
27.149999618530273
38.30500030517578
33.599998474121094
40.64999961853027
26.5
33.53999900817871
29.050000190734863
33.21999931335449
27.51

## ADT Calculation

In [6]:
fields = ["FN_E_T", "FS_W_T", "SN_E_T", "SS_W_T", "CLADT"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[4] = equations.calc_adt(row[0], row[1], row[2], row[3])
        print(row[4])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

8308
2477
2597
7210
1856
1400
6368
3636
3800
1744
691
1132
1387
1794
864
1765
1986
12391
2025
1218
0
1311
2527
871
606
4261
5740
277
5908
419
321
863
1598
1026
1923
2297
696
803
730
705
2014
2170
2459
842
1279
972
399
419
1724
540
2549
1002
1609
390
1006
2297
1150
1142
1358
1337
2282
4100
592
337
1001
5323
589
1332
5830
1332
2320
1495
5619
1468
218
316
3190
2068
2145
618
7081
7210
1747
2003
3464
1928
1765
1579
2205
406
1855
1335
1002
589
4043
2404
1001
3687
1461
257
3982
1105
378
1108
493
2040
6462
1174
12186
591
782
696
4231
7483
4923
1356
2614
659
1084
2028
4789
942
1459
616
658
654
1329
473
311
1002
588
451
633
1384
633
1180
830
1001
1696
701
801
3724
1227
1177
688
804
2422
12899
1518
804
1777
1125
2133
1976
1001
1651
7580
4891
556
1411
2767
1298
426
957
992
1394
1118
6511
366
674
1741
2459
2567
266
1001
1001
1020
692
439
1125
264
762
413
540
0
492
1075
816
1153
1730
1001
488
1154
683
4666
6451
1163
192
165
2646
1164
1609
504
3281
268
532
1308
788
629
189
4307
520
1094
506
4670
285


## Rank Score

In [7]:
fields = ["SP", "CL85", "CLADT", "KSI", "THYA", "FYA", "SW", "CDP", "DA", "SR", "BR", "Park", "OSF", "SWI", "RS"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        width_factor = 1 if row[13] < 36 else 0
        sw_factor = 0.5 if row[6] == "No" else 0
        cd_factor = 0.5 if row[7] == "Yes" else 0
        
        sf_int = []
        for sf in row[9:13]:
            sf_int.append(1 if sf == "Yes" else 0)
        sf_points = equations.calc_sf_points(sf_int, width_factor)

        da = 1 if row[8] == "Yes" else 0

        row[14] = float(equations.rank_score(row[0], row[1], row[2], row[3], row[4], row[5], sw_factor + cd_factor, da, sf_points))
        print("Rank Score", row[14] - 5)
        print("")
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

CK:  0
CL:  18.27
CM:  20
CO:  12
Rank Score -5.0

CK:  7.859996795654297
CL:  3.6925
CM:  20
CO:  30
Rank Score 61.55249679565429

CK:  8.639991760253906
CL:  3.9925
CM:  10
CO:  0
Rank Score 22.632491760253906

CK:  0
CL:  15.525
CM:  0
CO:  30
Rank Score -5.0

CK:  9.150009155273438
CL:  2.14
CM:  20
CO:  0
Rank Score 31.290009155273438

CK:  7.049995422363281
CL:  1.0
CM:  20
CO:  0
Rank Score 28.04999542236328

CK:  17.490005493164062
CL:  13.42
CM:  20
CO:  0
Rank Score 50.910005493164064

CK:  12.764991760253906
CL:  6.59
CM:  20
CO:  0
Rank Score 39.35499176025391

CK:  29.535003662109375
CL:  7.0
CM:  17
CO:  0
Rank Score 53.535003662109375

CK:  9.360008239746094
CL:  1.86
CM:  20
CO:  26
Rank Score 57.22000823974609

CK:  9.075004577636719
CL:  0
CM:  20
CO:  0
Rank Score -5.0

CK:  18.555004119873047
CL:  0.33
CM:  17
CO:  0
Rank Score 35.885004119873045

CK:  39.89999771118164
CL:  0.9675
CM:  20
CO:  0
Rank Score 60.86749771118164

CK:  23.25
CL:  1.985
CM:  20
CO:  0
Ran